# Seeme Quick Guide - Fastai v1

This is a quick guide to deploy your trained models using [SeeMe.ai](https://seeme.ai).

In this guide, we will create, deploy and use a simple Cats and Dogs classifier. 

The focus is getting you up and running with deploying, using and sharing your AI models. For the ins and outs of training AI models with Fast.ai, we recommend you follow the excellent [Fast.ai courses](fast.ai).

# 0. Disclaimer

* SeeMe.ai is currently in development and things might break:
    * For the beta period, you can host one model with up to 500 predictions for free;
    * We are working on a fair and sustainable pricing (contact [iwantto@seeme.ai](mailto:iwantto@seeme.ai) for more info);
    * We rely on your code of honor for fair use.

# 1. Setup

## 1.1 Where to run your Jupyter Notebooks

Use your preferred platfrom to run your [Jupyter Notebooks](https://jupyter.org/): 

* [Paperspace](https://paperspace.com)
* [Google Colab](https://colab.research.google.com/)
* [The official Fast.ai Docker images](https://hub.docker.com/u/fastdotai)
* [Fast.ai Docker from SeeMe.ai](https://hub.docker.com/repository/docker/seemeai/fastai2)
* [Azure](https://azure.microsoft.com)
* ...

In [ ]:
import torch
print(f"PyTorch version: {torch.__version__}")

In [ ]:
import fastai
print(f"Fastai version: {fastai.__version__}")

## 1.2 SeeMe.ai Python SDK

Install the [SeeMe SDK](https://pypi.org/project/seeme/) from the command line:

```bash
$ pip install --upgrade --no-cache-dir seeme
```

or in your Notebook:

In [2]:
# Install the SeeMe SDK from your notebook.
!pip install --upgrade --no-cache-dir seeme

Requirement already up-to-date: seeme in /opt/conda/lib/python3.7/site-packages (0.9.1)


# 2. Train a model

## 2.1 Train our Cats vs dogs model

Here we will train a basic Cats vs Dogs model, using the defaults from fastai2.

In [ ]:
# Import the vision modules
from fastai.vision import *

In [ ]:
# get the `catsdogs` dataset
path = untar_data(URLs.DOGS)

In [ ]:
# have a look at the folder
path.ls()

In [ ]:
# Fix the random seed
np.random.seed(12)

In [ ]:
# Set the image and batch size
image_size = 224
batch_size = 20

In [ ]:
# Create the dataloaders
# for more details check the fast.ai courses
data = ImageDataBunch.from_folder(path, 
                                  ds_tfms=get_transforms(), 
                                  size=image_size, 
                                  bs=batch_size
                                  ).normalize(imagenet_stats)

In [ ]:
# Create your model
learn =cnn_learner(data, models.resnet50, metrics=error_rate)

In [ ]:
# Fine tune your model
learn.fit_one_cycle(5)

## 2.2 Test our model locally

Let's see what our model says about one of the test pictures it has never seen.

In [ ]:
image_location = path/"test1/10789.jpg"

In [ ]:
# Find an image to test our model with
img = open_image(image_location)

In [ ]:
# Pretty clear for us humans
img

In [ ]:
# Let's see what our model thinks
learn.predict(img)

# 3. Deployment - SeeMe.ai

Once you have trained your model, it's time to get it into production.

## 3.1 Export your model

In [ ]:
# Put your model in eval model
learn.model.eval();

In [ ]:
# Export your model
learn.export()

## 3.2 Setup SeeMe SDK: Import, Register, Login

In [ ]:
# Import the SeeMe client
from seeme import Client

In [ ]:
# Create a SeeMe client
client = Client()

In [ ]:
# If you haven't already registered, follow these steps:
my_password =  # example: "supersecurepasswordfromonepassword"
my_username =  # example: "janvdp"
my_email =   # example: "jan@seeme.ai"
my_firstname =   # example: "Jan"
my_name =  # example: "Van de Poel"

client.register(username=my_username, email=my_email, password=my_password, firstname=my_firstname, name=my_name)

In [ ]:
# Once you have registered, you can log in:
client.login(my_username, my_password)

## 3.3 SeeMe.ai supports different applications and versions

With SeeMe.ai we support different types of AI applications with different frameworks and framework versions. 
To have a look at the supported applications and their capabilities, you can print all the supported applications in the cell below.

We currently support different versions of Fastai and object detection, but more frameworks are coming.

In [ ]:
from pprint import pprint
pprint(client.applications)

In [ ]:
# SeeMe support different types of applications, with different frameworks and versions.

# Get the applicationID for your framework (version).
application_id = client.get_application_id(
    base_framework="pytorch",
    framework="fastai",
    base_framework_version=str(torch.__version__),
    framework_version=str(fastai.__version__),
    application="image_classification"
  )

In [ ]:
# Create model with name and description

model_name = "My first model"
description = "Created to be used..."
classes = "Cats and dogs"

my_model = client.create_full_model({
    "name": model_name,
    "description": description,
    "classes": classes,
    "application_id": application_id
})

## 3.4 Upload your model

In [ ]:
# Deploy your exported model.
client.upload_model(my_model["ID"], str(path))

# By default this takes the export.pkl created by learn.export(), but you can simply pass in a filename as well.

## 3.5 Add a logo

In [ ]:
# Upload a logo for your model (optional)
client.upload_logo(my_model["ID"], folder=str(path/"test1"), filename="9.jpg")

# 4. Use your model

Once your model is deployed, you can use it in a number of ways:

- [Python SDK](https://pypi.org/project/seeme/)
- [web app](https://app.seeme.ai)
- [iOS - App Store](https://apps.apple.com/us/app/id1443724639)
- [Android - Play Store](https://play.google.com/store/apps/details?id=ai.seeme)

## 4.1 Python SDK

You can now use the [Python SDK](https://pypi.org/project/seeme/) to make predictions from basically anywhere, provided you have:

- SeeMe SDK installed
- Login credentials
- The ID of a deployed model
- An image to classiy

😃

In [ ]:
# make sure you are logged in.

In [ ]:
# Here, we will use the Python SDK to classify our test image
result = client.inference(my_model["ID"], image_location)

In [ ]:
# Print out the results
print(result["prediction"])
print(result["confidence"])

Make sure to check if they match the predictions from our model in the notebook.

## 4.2 SeeMe.ai web app

You can open the web app via [app.seeme.ai](https://app.seeme.ai)

Log in if you need to with the credentials used to register (my_username, my_password).

If you've followed the steps above without changing anything, this is what you will see after logging in.

Click on the model to start making predictions.

![seeme-ai-your-first-model-cats-dogs](images/seeme-ai-first-model-cats-dogs.png)

Here is what the detail screen looks like:

![SeeMe.ai first model detail screen](images/seeme-ai-model-detail-screen.png)

Next:

- click on `select image`
- find an image you would like to classify
- click on analyze
- Look at `result` and `confidence` to see what the prediction is.

![SeeMe.ai model prediction example](images/seeme-ai-model-prediction-example.png)

## 4.3 iOS/Android

You can also access all models - yours, shared, and public - on our native mobile apps. You can install them from the [iOS App Store](https://apps.apple.com/us/app/id1443724639) and [Android Play Store](https://play.google.com/store/apps/details?id=ai.seeme).

Once you have logged in, you see the overview of models:

![SeeMe.ai mobile list of models](images/seeme-ai-mobile-list-of-models.png)

On the model detail, you can take pictures with the camera or select from the gallery:

![SeeMe.ai model detail](images/seeme-ai-mobile-model-detail.png)

And see what your model thinks:

![SeeMe.ai model prediction](images/seeme-ai-model-prediction.png)

Once your model has made a prediction, you will see a green button name "Action". When clicking that button, you have a number of choices:

* search Google for your prediction
* search Wikipedia for your prediction
* Report a wrong prediction

![SeeMe.ai follow up action](images/seeme-ai-mobile-action-selection.png)

On [iOS](https://apps.apple.com/us/app/id1443724639), we have added automatic conversion to [CoreML](https://developer.apple.com/documentation/coreml) and [ONNX](https://onnx.ai/), when you upload your *.pkl. (Note: only for Fast.ai v2 at the moment).

This allows you to install and use the model on your device, even when you are offline. 

When the model is available to be installed, you will see the install button on the top right:

![SeeMe.ai install model offline](images/seeme-ai-mobile-model-install-offline.png)

Once installed, you can still switch between using the offline or online version of your model:

![SeeMe.ai switch between online and offline model](images/seeme-ai-mobile-switch-online-offline.png)

The screenshots above are all taken on iOS, however, on [Android](https://play.google.com/store/apps/details?id=ai.seeme) all these features work similarly, with one exception for the moment. On Android offline models are still in development.

## 4.4 Other platforms

If there is a particular platform you would live to use to make predictions and the above options don't work for you, we would be happy to hear from you. We offer API and Docker support as well.

# 5 Share your model

Once you have tested your model, it is time to share it with friends.

Go back to the home page, and click the `edit` icon.

![SeeMe.ai edit your model](images/seeme-ai-first-model-cats-dogs-edit.png)

You will go to the model detail screen:

![SeeMe.ai Model detail](images/seeme-ai-model-detail.png)

There you can invite people by entering their email address.

Once invited, they will receive an email to either register (if that email is not yet associated to an account) or to notify of your model being shared with them.